# Docking fragments to PHIP2 site 1

This content is by Nathan Lim and David Mobley, originally writen/adapted for a course (and available on GitHub at https://github.com/MobleyLab/drug-computing/blob/master/uci-pharmsci/lectures/docking_scoring_pose/OEDocking.ipynb) with heavy influence from the OpenEye documentation. It was adapted by David Mobley to the present PHIP2 Stage 2 challenge, and also utilizes examples I prepared for SAMPL5. https://github.com/samplchallenges/SAMPL6/blob/master/host_guest/GenerateInputs.ipynb

In [1]:
#Import our required openeye modules
from openeye import oechem, oedocking, oeomega
import os

In [2]:
# Load protein as preparation for making a "receptor" for docking
#protein_file = '../../../PHIPA_C2_Apo.pdb'
protein_file = '5enh_protein.pdb'
ligand_file = '5enh_ligand.pdb'

protein = oechem.OEGraphMol()
ligand = oechem.OEGraphMol()
ifile = oechem.oemolistream(protein_file)
oechem.OEReadMolecule(ifile, protein)
ifile.close()
ifile = oechem.oemolistream(ligand_file)
oechem.OEReadMolecule(ifile, ligand)
ifile.close()
#ifile = 'PHIPA_C2_DU.oedu'
#du = oechem.OEDesignUnit()
#oechem.OEReadDesignUnit(ifile, du)
#du.GetProtein(protein)

# INPUT BINDING SITE LOCATION
siteloc = oechem.OEFloatArray(3)
siteloc[0] = -19.150
siteloc[1] = -12.842
siteloc[2] = 24.7000

# Receptor file name
receptor_file = '5enh_prepped.oeb'
#receptor_file = 'PHIP2_apo_prepped.oeb'
receptor = oechem.OEGraphMol()

# Prep receptor
if not os.path.isfile(receptor_file):
    oedocking.OEMakeReceptor(receptor, protein, ligand)
    #oedocking.OEMakeReceptor(receptor, protein, siteloc[0], siteloc[1], siteloc[2])
    oedocking.OEWriteReceptorFile(receptor, receptor_file)
else:
    #Read in our receptor from disc
    if not oedocking.OEReadReceptorFile( receptor, receptor_file ):
    # raise an exception if the receptor file cannot be read
        raise Exception("Unable to read receptor from {0}".format( receptor_file ))

In [3]:
#Set the docking method and other paramters
# Note: Chemgauss4 is the scoring function for FRED
dock_method = oedocking.OEDockMethod_Chemgauss4
dock_resolution = oedocking.OESearchResolution_Default
sdtag = oedocking.OEDockMethodGetName( dock_method )

#Generate our OEDocking object
dock = oedocking.OEDock( dock_method, dock_resolution)

#Initialize the OEDocking by providing it the receptor
if not dock.Initialize(receptor):
    # raise an exception if the receptor cannot be initialized
    raise Exception("Unable to initialize Docking with {0}".format(receptor_file))

Now that we have initialized our OEDocking object with our receptor, let's write a function that will take in the following input parameters:
    - dock: OEDock object
    - sdtag: string representing the name of the docking method
    - numpose: int with the number of poses to generate for each ligand
    - mcmol: multicomformer molecule

In [4]:
def dock_molecule( dock: "OEDock", sdtag: str, num_poses: int, mcmol ) -> tuple:
    ''' Docks the multiconfomer molecule, with the given number of poses
        Returns a tuple of the docked molecule (dockedMol) and its score
        i.e. ( dockedMol, score )
    '''
    dockedMol = oechem.OEMol()

    #Dock the molecule into a given number of poses
    res = dock.DockMultiConformerMolecule(dockedMol, mcmol, num_poses)
    
    if res == oedocking.OEDockingReturnCode_Success:
        
        #Annotate the molecule with the score and SDTag that contains the docking method
        oedocking.OESetSDScore(dockedMol, dock, sdtag)
        dock.AnnotatePose(dockedMol)
        score = dock.ScoreLigand(dockedMol)
        oechem.OESetSDData(dockedMol, sdtag, "{}".format(score))
        return dockedMol, score
    
    else:
        # raise an exception if the docking is not successful
        raise Exception("Unable to dock ligand {0} to receptor".format( dockedMol ))

With the docking function written, we can then loop over our 3D molecules and dock them to the given receptor

In [5]:
# Read input molecule SMILES
ligand_file = '../../../Stage-2-input-data/site-1_fragment-hits.csv'
file = open(ligand_file,'r')
text = file.readlines()
file.close()
ligands = {}

omega = oeomega.OEOmega()
omega.SetMaxConfs(500)
# Temporary hack for F501 and F179, see https://github.com/samplchallenges/SAMPL7/issues/77
omega.SetStrictStereo(False)

for line in text:
    tmp = line.split(',')
    mol = oechem.OEMol()
    mol.SetTitle(tmp[0])
    smi = tmp[1].strip()
    
    # Parse SMILES to OEMol, generate conformers
    oechem.OEParseSmiles(mol, smi)
    status = omega(mol)
    if not status:
        print(f"Error running omega on {tmp[0]}.")
    
    # Store smiles string, mol w conformers to dictionary as tuple
    ligands[tmp[0].strip()] = ( smi , mol)

In [6]:
#Define how many docked poses to generate per molecule
num_poses = 1

outdir = 'poses'
if not os.path.isdir(outdir): os.mkdir(outdir)

#Read in our 3D molecules 
for lig_code in ligands:
    #Call our docking function
    dockedMol, score = dock_molecule( dock, sdtag, num_poses, ligands[lig_code][1] )
    print("{} {} score = {:.4f}".format(sdtag, dockedMol.GetTitle(), score))

    # Write output; as per challenge instructions each fragment to separate file
    ofs = oechem.oemolostream(os.path.join(outdir, f'PHIP2-{lig_code}-1.sdf'))
    oechem.OEWriteMolecule(ofs, dockedMol)
    ofs.close()

Chemgauss4 F5 score = -6.9151
Chemgauss4 F11 score = -6.7192
Chemgauss4 F13 score = -6.8085
Chemgauss4 F14 score = -6.6209
Chemgauss4 F95 score = -7.8034
Chemgauss4 F96 score = -6.9841
Chemgauss4 F126 score = -7.1434
Chemgauss4 F179 score = -7.6280
Chemgauss4 F199 score = -6.0217
Chemgauss4 F205 score = -6.1165
Chemgauss4 F217 score = -8.1661
Chemgauss4 F229 score = -8.8128
Chemgauss4 F274 score = -6.9522
Chemgauss4 F275 score = -6.4773
Chemgauss4 F309 score = -6.0000
Chemgauss4 F362 score = -5.9153
Chemgauss4 F366 score = -7.2678
Chemgauss4 F367 score = -7.9953
Chemgauss4 F368 score = -6.5665
Chemgauss4 F369 score = -7.0030
Chemgauss4 F374 score = -7.6004
Chemgauss4 F389 score = -7.3000
Chemgauss4 F393 score = -6.2913
Chemgauss4 F421 score = -7.6960
Chemgauss4 F467 score = -7.0489
Chemgauss4 F488 score = -8.6425
Chemgauss4 F501 score = -7.2978
Chemgauss4 F503 score = -7.0994
Chemgauss4 F529 score = -6.3791
Chemgauss4 F535 score = -6.5930
Chemgauss4 F558 score = -8.1250
Chemgauss4 F579

In [8]:
# Make protein files to go along with submisssion
import shutil
for lig_code in ligands:
    outname = os.path.join(outdir, f'PHIP2-{lig_code}-1.pdb')
    shutil.copy(protein_file, outname)

# Acknowledgements

- Authors: Meghan Osato, Nathan M. Lim, David L. Mobley